In [1]:
import pandas as pd
import statistics
import matplotlib.pyplot as plt


In [20]:
# Enter the path to the tsv file with the rating
path_tsv = "/Users/camille.brianceau/aramis/reproducibility-reviews/results23/rating_repro_reviews.csv"
df_rating_1 = pd.read_csv(path_tsv, sep = "\t", index_col=False, header= None)


In [27]:
df_review1 = df_rating_1.loc[:,2:11]
df_review2 = df_rating_1.loc[:,11:20]
df_review3 = df_rating_1.loc[:,20:29]


In [29]:
print(df_review1.loc[:,3])

0      Put 1 if the reviewer has commented on at leas...
1                                  Models and algorithms
2                                                    NaN
3                                                    NaN
4                                                    NaN
                             ...                        
727                                                  NaN
728                                                  NaN
729                                                  NaN
730                                                  NaN
731                                                  NaN
Name: 3, Length: 732, dtype: object


In [15]:
# Number of reviewer who has commented on at least one of models and algorithms for 
print(df_rating_1.index.values)
print(df_rating_1.columns.values)



[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

In [19]:
df_rating_1.index.get_level_values(0)
df_rating_1.set_index([0,1])

ValueError: Length mismatch: Expected 732 rows, received array of length 2